In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.default'] = 'regular'
import numpy as np



In [6]:
# Load the IC50 data
IC50s = pd.read_csv('Final IC50 dataset.csv', delimiter= ';')

def parse_protocol(protocol):
    if protocol == 'CiPA':
        return True, True
    if protocol == 'Pharm':
        return False, False

def parse_rescaling(drug, concentration, CiPA_hERG, CiPA_CaV):
    IC50s_filtered = IC50s[IC50s['Drug'] == drug]

    if CiPA_hERG:
        #print('hERG IC50 : ' + str(np.array(IC50s_filtered['hERG IC50 CiPA'])) + ' / ' + str(np.array(IC50s_filtered['hERG h CiPA'])))
        hERG_IC50 = np.array(IC50s_filtered['hERG IC50 CiPA'])
        hERG_h = np.array(IC50s_filtered['hERG h CiPA'])
    else:
        #print('hERG IC50 : ' + str(np.array(IC50s_filtered['hERG IC50 Pharm'])) + ' / ' + str(np.array(IC50s_filtered['hERG h Pharm'])))
        hERG_IC50 = np.array(IC50s_filtered['hERG IC50 Pharm'])
        hERG_h = np.array(IC50s_filtered['hERG h Pharm'])

    if CiPA_CaV:
        #print('CaV IC50 : ' + str(np.array(IC50s_filtered['CaV IC50 CiPA'])) + ' / ' + str(np.array(IC50s_filtered['CaV h CiPA'])))
        CaV_IC50 = np.array(IC50s_filtered['CaV IC50 CiPA'])
        CaV_h = np.array(IC50s_filtered['CaV h CiPA'])
    else:
        #print('CaV IC50 : ' + str(np.array(IC50s_filtered['CaV IC50 Pharm'])) + ' / ' + str(np.array(IC50s_filtered['CaV h Pharm'])))
        CaV_IC50 = np.array(IC50s_filtered['CaV IC50 Pharm'])
        CaV_h = np.array(IC50s_filtered['CaV h Pharm'])

    # Compute current block
    ikr_rescale = 1 / (1 + np.power(concentration / hERG_IC50, hERG_h))
    ical_rescale = 1 / (1 + np.power(concentration / CaV_IC50, CaV_h))

    return ikr_rescale, ical_rescale

In [8]:
def curate_df(df, protocol):
    # Replace NaN cons by 0
    df['Nominal concentration 2'] = df['Nominal concentration 2'].fillna(0)
    df['Measured concentration 2'] = df['Measured concentration 2'].fillna(0)

    # Delete the rows with two drugs
    df = df.drop(df[df['Nominal concentration 2'] > 0].index)
    # Delete the rows with no drug
    df = df.drop(df[df['Nominal concentration 1'] == 0].index)

    # Group by drug effect
    df_clean = df.groupby(['Drug 1', 'Nominal concentration 1', 'Nominal concentration 2'], as_index = False).mean()
    SEMs = df.groupby(['Drug 1', 'Nominal concentration 1', 'Nominal concentration 2'], as_index = False).sem()
    df_clean['SEM'] = SEMs['DAPD']

    # Clean columns
    df_clean = df_clean.drop(columns = ['Unnamed: 0', 'Trabecula', 'APD90'])


    # Recompute the IKr and ICaL block
    ikr_blocks = np.zeros(len(df_clean))
    ical_blocks = np.zeros(len(df_clean))

    for i in range(len(df_clean)):
        CiPA_hERG, CiPA_CaV = parse_protocol(protocol)
        ikr_blocks[i], ical_blocks[i] = parse_rescaling(df_clean['Drug 1'][i], df_clean['Measured concentration 1'][i], CiPA_hERG, CiPA_CaV)

    df_clean['IKr block'] = ikr_blocks
    df_clean['ICaL block'] = ical_blocks

    df_clean.to_csv('Data for benchmark - ' + protocol + '.csv', sep = ',')
    return df_clean

# Load the whole dataset
CiPA_df = pd.read_csv('All APD90 and drug concentration data - CiPA.csv', delimiter = ';')
Pharm_df = pd.read_csv('All APD90 and drug concentration data - Pharm.csv', delimiter = ';')

# Curate
CiPA_clean_df = curate_df(CiPA_df, 'CiPA')
Pharm_clean_df = curate_df(Pharm_df, 'Pharm')